In [1]:
import sys

from pathlib import Path
main_path = Path().absolute().parent
data_path = main_path / 'data'
setting_path = main_path / 'setting_files'

sys.path.append(str(main_path))

import yaml
import pytorch_lightning as pl
from pytorch_lightning import seed_everything

from src.nlu_models import NLUModel
from src.nlu_utils import NLUDataModule

with (setting_path / 'eval_settings.yml').open('r') as file:
    settings = yaml.load(file, Loader=yaml.FullLoader)

data_module = NLUDataModule(
    train_path=data_path / settings['train_file'], 
    valid_path=data_path / settings['valid_file'],
    test_path=data_path / settings['test_file'],
    labels_path=data_path / settings['labels_file'],
    batch_size=settings['batch_size'], 
    max_len=settings['max_len'],
    num_workers=settings['num_workers'],
    seed=settings['seed']
)

hparams = {
    'stage': settings['stage'],
    'model_path': settings['model_path'], 
    'intent_size': len(data_module.intents2id), 
    'tags_size': len(data_module.tags2id), 
    'max_len': settings['max_len'],
    'lr': settings['lr'],
    'weight_decay_rate': settings['weight_decay_rate'],
    'loss_type': settings['loss_type'],
    'multigpu': True if settings['n_gpus'] > 1 else False
}

checkpoint_path = str(main_path / 'checkpoints' / settings['exp_name'] / 'best_model.ckpt')

model = NLUModel.load_from_checkpoint(checkpoint_path)
model.eval()

seed_everything(seed=settings['seed'])
trainer = pl.Trainer(
    gpus=settings['n_gpus'], 
    max_epochs=settings['n_epochs'], 
    num_sanity_val_steps=settings['num_sanity_val_steps'],
    deterministic=True,
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

In [ ]:
# results = trainer.test(model, ckpt_path=checkpoint_path, datamodule=data_module)

In [2]:
from src.nlu_utils import NLUTokenizer

tokenizer = NLUTokenizer()

In [3]:
def predict(model, tokenizer, text):
    bert_encodes = tokenizer(
        text, 
        add_special_tokens=True, 
        truncation=True, 
        max_length=256,
        return_tensors='pt'
    )
    output = model.predict(**bert_encodes)
    return output

In [4]:
data = {"text": "-- dhaka newsroom 880-2-506363", "entities": [[3, 17, "ORG"]], "intent": "None"}
text = data['text']

In [5]:
data_module.prepare_data()
train_dataset = data_module.create_dataset(data_module.train_data)

loading: 100%|██████████| 2996/2996 [00:00<00:00, 200263.51it/s]


In [6]:
import torch

In [7]:
batch = {k: torch.as_tensor([train_dataset[0][k]]) for k in ['input_ids', 'token_type_ids', 'attention_mask']}

In [8]:
o = model(**batch)

In [9]:
o

{'tags': tensor([[ 3.4260, -1.0233, -4.5061, -0.8361,  0.3515, -1.5198, -1.2052, -2.1704,
          -0.9339, -1.4328, -0.7987, -0.9890, -1.0170, -0.4447, -0.3669,  0.1521,
           0.9416,  0.2291,  0.6546,  0.1392, -0.5912, -0.5209, -0.8010],
         [ 3.4260, -1.0233, -4.5061, -0.8361,  0.3515, -1.5198, -1.2052, -2.1704,
          -0.9339, -1.4328, -0.7987, -0.9890, -1.0170, -0.4447, -0.3669,  0.1521,
           0.9416,  0.2291,  0.6546,  0.1392, -0.5912, -0.5209, -0.8010],
         [ 3.4260, -1.0233, -4.5061, -0.8361,  0.3515, -1.5198, -1.2052, -2.1704,
          -0.9339, -1.4328, -0.7987, -0.9890, -1.0170, -0.4447, -0.3669,  0.1521,
           0.9416,  0.2291,  0.6546,  0.1392, -0.5912, -0.5209, -0.8010],
         [ 3.4260, -1.0233, -4.5061, -0.8361,  0.3515, -1.5198, -1.2052, -2.1704,
          -0.9339, -1.4328, -0.7987, -0.9890, -1.0170, -0.4447, -0.3669,  0.1521,
           0.9416,  0.2291,  0.6546,  0.1392, -0.5912, -0.5209, -0.8010],
         [ 3.4260, -1.0233, -4.5061, -0.

In [7]:
o = predict(model, tokenizer, text)
o

{'tags': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'intent': tensor([0])}

In [17]:
bert_encodes = tokenizer(
    text, 
    add_special_tokens=True, 
    truncation=True, 
    max_length=256,
    return_tensors='pt'
)

In [10]:
import torch.nn as nn
ce = nn.CrossEntropyLoss(reduction='none')

In [11]:
logits = torch.FloatTensor([
    [9.3, -1.3, 0.14],
    [1.5, 7.3, 5.4],
    [9.3, -1.3, 0.14],
])
labels = torch.LongTensor([0, 0, 1])

In [33]:
ce_loss = ce(logits, labels)
pt = torch.exp(-ce_loss)
gamma = 300
alpha = 1
loss = alpha * (1-pt)**gamma * ce_loss

print(ce_loss.numpy().round(3))
print(loss.numpy().round(3))

[ 0.     5.942 10.6  ]
[ 0.     2.699 10.521]


In [18]:
o = model(**bert_encodes)

In [19]:
o

{'tags': tensor([[ 5.1541, -1.0479, -5.5954, -0.7491,  0.2225, -1.3268, -1.0800, -2.2567,
          -1.1802, -1.4342, -1.0662, -1.1117, -1.1067, -0.4033, -0.5380,  0.2236,
           0.8920,  0.1433,  0.0643,  0.4633, -0.2504, -0.2810, -1.0804],
         [ 5.1541, -1.0479, -5.5954, -0.7491,  0.2225, -1.3268, -1.0800, -2.2567,
          -1.1802, -1.4342, -1.0662, -1.1117, -1.1067, -0.4033, -0.5380,  0.2236,
           0.8920,  0.1433,  0.0643,  0.4633, -0.2504, -0.2810, -1.0804],
         [ 5.1541, -1.0479, -5.5954, -0.7491,  0.2225, -1.3268, -1.0799, -2.2567,
          -1.1802, -1.4342, -1.0662, -1.1117, -1.1067, -0.4033, -0.5380,  0.2236,
           0.8920,  0.1433,  0.0643,  0.4633, -0.2504, -0.2810, -1.0804],
         [ 5.1541, -1.0479, -5.5954, -0.7491,  0.2225, -1.3268, -1.0799, -2.2567,
          -1.1802, -1.4342, -1.0662, -1.1117, -1.1067, -0.4033, -0.5380,  0.2236,
           0.8920,  0.1433,  0.0643,  0.4633, -0.2504, -0.2810, -1.0804],
         [ 5.1541, -1.0479, -5.5954, -0.